In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("spamm.csv")

In [ ]:
df.head()

In [ ]:
df.Category = df.Category.map({'ham':0, 'spam':1})
df.head()

In [ ]:
df.Category.value_counts()

In [ ]:
df_spam = df[df.Category==1]
df_spam.shape

In [ ]:
df_ham_small = df[df.Category==0].sample(1000)
df_ham_small.shape

In [ ]:
df_small = pd.concat([df_ham_small,df_spam])
df_small.Category.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_small.Message,df_small.Category, test_size=0.2, random_state=5)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(texts, labels):
    encodings = tokenizer(texts, padding='max_length', max_length=128, truncation=True, return_tensors='pt')
    return encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels, dtype=torch.float)

tokenize_function(["Hurry up, click here", "I will see you tomorrow"], [1,0])

In [ ]:
X_train.value_counts()[:2]

In [ ]:
y_train.head(3)

In [ ]:
train_input_ids, train_attention_mask, train_labels = tokenize_function(X_train.values.tolist(), y_train.values.tolist())
val_input_ids, val_attention_mask, val_labels = tokenize_function(X_test.values.tolist(), y_test.values.tolist())

In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = torch.utils.data.TensorDataset(val_input_ids, val_attention_mask, val_labels)

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
bert = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        for param in self.bert.parameters():
            param.requires_grad = False # Freeze all BERT layers
        
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sentence_embedding = bert_output.last_hidden_state[:,0,:]
        return self.classifier(sentence_embedding)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = SentimentClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
epochs = 1

# Training loop
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch, (input_ids, attention_mask, labels) in enumerate(train_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Batch: {batch}, Epoch: {epoch}, Loss:  {loss.item():0.2f}")
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss}')

In [ ]:
# Evaluation
model.eval()
total_val_loss = 0
correct_predictions = 0

with torch.no_grad():
    for input_ids, attention_mask, labels in val_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(input_ids, attention_mask).squeeze()
        loss = criterion(outputs, labels.view_as(outputs))
        total_val_loss += loss.item()

        preds = (outputs > 0.5).float()
        correct_predictions += torch.sum(preds == labels)
        
avg_val_loss = total_val_loss / len(val_loader)
val_accuracy = correct_predictions.double() / len(val_dataset)
print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy:.4f}')    

In [ ]:
def predict(model, text, max_length=128):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Tokenize input text
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    model = model.to(device)
    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask).squeeze()
        prediction = (output > 0.5).float().item()
        return 'spam' if prediction == 1 else 'ham'